# BoolForge Tutorial #7: Dynamics of Boolean networks

In this tutorial, we study the *dynamics* of Boolean networks.
Building on the construction and structural analysis from previous tutorials, we now focus on how Boolean networks evolve over time and how their long-term behavior can be characterized.

You will learn how to:
- simulate Boolean network dynamics under different updating schemes,
- compute and classify attractors,
- analyze basins of attraction, and
- relate network structure to dynamical behavior.

## 0. Setup

In [ ]:
import boolforge
import numpy as np
import pandas as pd


## 1. State space of a Boolean network

A Boolean network with $N$ nodes defines a dynamical system on the discrete state space $\{0,1\}^N.$

Each state is a binary vector
$$
\mathbf x = (x_0,\ldots,x_{N-1}) \in \{0,1\}^N,
$$
where $x_i$ denotes the state of node $i$.

The full state space consists of $2^N$ states. 
For small $N$, the entire state space can be enumerated and analyzed explicitly.

We will use a small Boolean network as a running example throughout this tutorial.

In [ ]:
string = '''x = y
y = x OR z
z = y'''

bn = boolforge.BooleanNetwork.from_string(string, separator='=')
print("Variables:", bn.variables)
print("N:", bn.N)
print('bn.I',bn.I)
print('bn.F',bn.F)


All state vectors in this tutorial will follow the variable order given by `bn.variables`. 
For small networks, we can explicitly enumerate all states in $\{0,1\}^N$.

In [ ]:
all_states = boolforge.get_left_side_of_truth_table(bn.N)
pd.DataFrame(all_states,columns = bn.variables)


Each state is represented as a numpy.array of length N, and we see the decimal representation of each state on the left of the displayed pandas.DataFrame.

## 2. Short- and long-term dynamics

The update rules, encoded in `bn.F`, describe the dynamic transitions of the Boolean network. 

Under a *synchronous update* scheme, all nodes are updated simultaneously.
This defines a deterministic update map
$$
F : \{0,1\}^N \to \{0,1\}^N,
\qquad
\mathbf x(t+1) = F(\mathbf x(t)).
$$

We can compute the next, synchronously updated state for any initial state:

In [ ]:
for state in all_states:
    print(state,'-->',bn.update_network_synchronously(state))


Each state has exactly one successor under synchronous updating.
As a result, the dynamics consist of transient trajectories leading into *steady states* and *cycles*, collectively known as *attractors*.

In our example, the Boolean network has two steady states ($x=0,y=0,z=0$ and $x=1,y=1,z=1$) as well as a cyclic attractor of periodicity 2 ($x=0,y=1,z=0 \leftrightarrow x=1,y=0,z=1$). 
All other states are transient and eventually end up at an attractor.

For any sufficiently small Boolean network (typically of size $N\lesssim 20$-$30$), the entire dynamics and long-term behavior can be computed exhaustively, allowing the full dynamics and long-term behavior to be computed exactly.

In [ ]:
dict_dynamics = bn.get_attractors_synchronous_exact()
dict_dynamics


This function returns a dictionary with five keys that collectively describe the network’s dynamics:

- `STG`: The *state transition graph* under synchronous updating. Each key is one of the $2^N$ states (in decimal representation), and the corresponding value is the next state after one synchronous update.
Use `boolforge.dec2bin` to convert states back to binary representation:

In [ ]:
for state in range(2**bn.N):
    next_state = dict_dynamics['STG'][state]
    print(state,'=',
          boolforge.dec2bin(state,bn.N),
          '-->',
          next_state,'=',
          boolforge.dec2bin(next_state,bn.N))

- `NumberOfAttractors`: The total number of distinct attractors in the network.
- `Attractors`: A list of the network attractors, each represented as a sequence of states in decimal form.
Again, `boolforge.dec2bin` can be used for readability:

In [ ]:
for attractor in dict_dynamics['Attractors']:
    print(f'Attractor of length {len(attractor)}:')
    for state in attractor:
        print(state,boolforge.dec2bin(state,bn.N))
    print()

- `AttractorDict`: While `STG` specifies the one-time step update of each state, this dictionary assigns every state to the index of the attractor that it eventually reaches (indexed as in `dict_dynamics['Attractors']`).
- `BasinSizes`: The *basin size* of each attractor, defined as the number of states that ultimately flow into it. The basin sizes always sum to $2^N$, since every state eventually transitions to an attractor.

For larger networks, exhaustive enumeration of the full state space quickly becomes computationally infeasible.
In such cases, *Monte Carlo simulation* can be used to identify all large attractors and to approximate the basin size distribution:

In [ ]:
dict_dynamics = bn.get_attractors_synchronous(nsim=100)
dict_dynamics


In addition to the keys described above, the resulting dictionary contains two further entries:
- `InitialSamplePoints`: The decimal representations of the nsim=100 randomly chosen initial states whose trajectories were followed until an attractor was reached.
- `NumberOfTimeouts`: The number of trajectories that failed to reach an attractor within the allotted time. By default, each trajectory is updated for at most `n_steps_timeout = 1000` steps (optional keyword), so this value is typically zero for well-behaved networks.

## 5. Attractors under synchronous updating

An *attractor* is a set of states that the system cannot leave once entered.
Under synchronous updating, attractors are directed cycles in the state transition graph.

In [ ]:
def find_attractor(bn, start_state):
    seen = {}
    state = start_state
    t = 0
    while state not in seen:
        seen[state] = t
        state = synchronous_update(bn, state)
        t += 1
    cycle_start = seen[state]
    cycle = [s for s, time in seen.items() if time >= cycle_start]
    return cycle

find_attractor(bn, states[0])


## 6. Basins of attraction

The basin of attraction of an attractor consists of all states that eventually lead to it.

In [ ]:
attractors = {}
for s in states:
    cycle = tuple(sorted(find_attractor(bn, s)))
    attractors.setdefault(cycle, []).append(s)

for cycle, basin in attractors.items():
    print("Attractor:", cycle)
    print("Basin size:", len(basin))